In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

from pandas.io.json import json_normalize
#used this as json file has nested dictionaries

import numpy as np

#used below to parse dates in publish_time column
d_parser = lambda x : pd.datetime.strptime(x[:10]+' '+x[11:19],"%Y-%m-%d %H:%M:%S")
df = pd.read_csv('C:\Python\Project\INvideos.csv', parse_dates = ['publish_time'],date_parser = d_parser)
        
#was unable to use above code to parse multiple date columns with different formats, hence used below lambda function
df["trending_date"] = df["trending_date"].apply(lambda x : pd.datetime.strptime(x,"%y.%d.%m"))
df1 = pd.read_json('C:\Python\Project\IN_category_id.json')
categories = json_normalize(df1["items"])
categories["id"] = categories["id"].astype('int64')
df = pd.merge(df,categories[["id","snippet.title"]],left_on =["category_id"],right_on = ['id'], how = 'left')
df.drop(columns = ["id"],inplace = True)
df.rename(columns = {"snippet.title":"category"},inplace = True)
#calculate time gap between video publish and trend time, time component available only in publish dt but not trend dt, 
#hence difference calculated in days only, type is int for easy calculations
df["time_to_trend"] = (df["trending_date"].astype('datetime64') - df["publish_time"].astype('datetime64')).dt.days
#print(df["time_to_trend"].head())


In [ ]:
df.shape

In [ ]:
df[df.duplicated()]

In [ ]:
filt = df["video_id"] == 'YAzs-timxnI'
df.loc[filt]

In [ ]:
#drop absolute duplicates
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique().sort_values(ascending=False)

In [ ]:
#investigate nulls in description
filt = df["description"].isna()
df.loc[filt]

In [ ]:
#investigate nulls in category column
#found there is no match for category for category_id 29 in any json,
#but figured out it is "Nonprofits & Activism", updating it below

In [ ]:
filt = df["category"].isna()

In [ ]:
df.loc[filt,'category'] = 'Nonprofits & Activism'

In [ ]:
#rerun uniquecount 
df.nunique().sort_values(ascending=False)

In [ ]:
#analyse why there are duplicate video_ids, notice count of unique video_ids is almost 50% less than total dataset
#also notice the #NAME? and #VALUE! which needs investigation as well
df.groupby("video_id")["video_id"].value_counts()

In [ ]:
#Pick any video id which has more than 1 entry
filt = df["video_id"] == 'zyvYkpoLeSU'

In [ ]:
df.loc[filt]
#entries have different trending_date and it appears that only one copy should be picked

In [ ]:
df["video_id"].str.len().unique()

In [ ]:
df["video_id"].str.len().value_counts()

In [ ]:
#before progessing further, investigate suspicious entries in video_id column
#looks like there is no pattern here, can be dropped from the main dataframe but stored in another 
#data frame for further analysis
filt = df["video_id"].str.len() < 11
df_investigateids = df.loc[filt]
df_investigateids

In [ ]:
df.drop(index = df[filt].index, axis = 0, inplace = True)
df["video_id"].str.len().value_counts()

In [ ]:
#just another way of finding max values
# temp = df.groupby("video_id")["trending_date"].max()
# temp
df[df.duplicated(['video_id','trending_date'])]

In [ ]:
#retain only rows with latest trending_date
filt = df.sort_values(by="trending_date").duplicated(subset=["video_id"], keep="last")
df.drop(index = df[filt].index, axis = 0,inplace = True)
#note the dropped rows can be used for further analysis on trending patterns

In [ ]:
#clean data
df.nunique()

In [ ]:
df.describe()

In [ ]:
anum = df["category"].value_counts(normalize = False).sort_values()
aper = df["category"].value_counts(normalize = True).mul(100).round(2)
acom = pd.concat([anum,aper],axis = 1, keys = ['Counts','%ages'])
print(acom)
bnum = pd.crosstab(df["category"],df["comments_disabled"])
bper = pd.crosstab(df["category"],df["comments_disabled"],normalize = True).mul(100).round(2).astype('str')+'%'
bcom = pd.concat([bnum,bper],axis = 1, keys = ['Counts','%ages']) 
print(bcom)
#indexes = bcom.index.get_level_values('category')
both = acom.merge(bcom, right_on = bcom.index.get_level_values('category'), left_on = acom.index, how = 'inner',sort = True)
both.sort_values(by = 'Counts',ascending = False)


# from IPython.display import display, HTML
# display(HTML(both.to_html()))



In [ ]:
import plotly.express as px
data = df["category"].value_counts().rename_axis('Category').reset_index(name = 'Counts')
#print(data,type(data),data.columns)
fig = px.pie(data,values = 'Counts',names = 'Category',width = 800, height = 400)
fig.update_layout(
    margin=dict(l=10, r=10, t=20, b=20, pad = 4), paper_bgcolor = "LightSteelBlue"
)
fig.show()

In [ ]:
filt = df["comments_disabled"] == True
df.loc[filt]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["category"].value_counts()

In [ ]:
print(df["category"].value_counts())
df["category"].value_counts().plot(kind = 'hist')

In [ ]:
cat_master_list = pd.Series(categories["id"].unique()).sort_values()

In [ ]:
cat_data_list = pd.Series(df["category_id"].unique()).sort_values()

In [ ]:
print(type(cat_master_list))
print(cat_data_list)

In [ ]:
cat_data_list.isin(cat_master_list)

In [ ]:
df["category"].value_counts(dropna = False)

In [ ]:
filt = df3["snippet.title"].isna()

In [ ]:
df3.loc[filt]

In [ ]:
#Top 10 trending videos
#top 5 words under tag for most disliked and most liked videos 
#most trending categories
#trending time for 
#summary = df.fillna('1').groupby(["category","channel_title"])[["views","likes","comment_count","dislikes"]].sum().sort_values(by =["views","likes","comment_count","dislikes"], ascending = False)
summary = df.fillna('1').groupby(["category","channel_title","video_id"])[["views","likes","comment_count","dislikes"]].sum().sort_values(by =["views"], ascending = False)

In [ ]:
summary

In [ ]:
summary["like%"] = (summary["likes"]/summary["views"]).mul(100).round(2).astype('str')+'%'

In [ ]:
summary["comment%"] = (summary["comment_count"]/summary["views"]).mul(100).round(2).astype('str')+'%'

In [ ]:
summary["dislike%"] = (summary["dislikes"]/summary["views"]).mul(100).round(2).astype('str')+'%'

In [ ]:
summary

In [ ]:
temp = pd.DataFrame(df["tags"].str.split(expand=True).stack().value_counts(), columns = ['counts'])

In [ ]:
temp1 = pd.DataFrame(df["tags"].str.split(expand=True).stack().value_counts(), columns = ['counts'])

In [ ]:
temp.info()

In [ ]:
comedy_filt = df["category"] == 'Comedy'
df.loc[comedy_filt]



In [ ]:
temp = pd.DataFrame(df["tags"].str.replace("\"",'').str.replace("|"," ").str.split(expand = True).stack().value_counts(), columns = ['countss'])

In [ ]:
filt = ((temp["countss"] >= 500) & (temp["countss"] <= 1000))

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

temp



In [ ]:
filt = df3["snippet.title"] == 'Travel & Events'

In [ ]:
df3.loc[filt]

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
plt.subplots(figsize=(8,8))
wordcloud = WordCloud(
                          background_color='white',
                          width=512,
                          height=384,max_words = 100
                         ).generate(" ".join(df["title"]))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')

plt.show()

In [ ]:
import sys
print(sys.executable)

In [ ]:
!c:\users\rajvenka1\appdata\local\programs\python\python37\python -m pip install wordcloud

In [ ]:
temp.index.sort_values()